In [1]:
import pandas as pd

In [3]:
dep_val = pd.read_csv("City_of_Seattle_Wage_Data.csv")

In [6]:
dep_val.columns

Index(['Department', 'Last Name', 'First Name', 'Job Title', 'Hourly Rate '], dtype='object')

In [9]:
dep_val['Last Name'].value_counts()

Last Name
Johnson      90
Nguyen       89
Smith        82
Lee          64
Anderson     59
             ..
Al-Rashid     1
Valencia      1
Van Thiel     1
Deittrick     1
Hui           1
Name: count, Length: 7744, dtype: int64

In [12]:
last_name_counts = dep_val['Last Name'].value_counts()

last_name_counts_filtered = last_name_counts[last_name_counts > 5]

In [13]:
last_name_counts_filtered

Last Name
Johnson      90
Nguyen       89
Smith        82
Lee          64
Anderson     59
             ..
Oliver        6
Porter        6
Ness          6
Zimmerman     6
Conrad        6
Name: count, Length: 242, dtype: int64

In [17]:
for department, data in dep_val.groupby("Department"):
    print(f"Group for department '{department}':")
    # print(data)

Group for department 'Arts & Culture':
Group for department 'City Auditor':
Group for department 'City Budget Office':
Group for department 'City of Seattle':
Group for department 'Civil Service Commissions':
Group for department 'Commnty Sfty and Comm Ctr Dept':
Group for department 'Community Police Commission':
Group for department 'Construction & Inspections':
Group for department 'Econ & Revenue Forecasts Dept':
Group for department 'Education & Early Learning':
Group for department 'Employees' Retirement System':
Group for department 'Ethics & Elections Commission':
Group for department 'Finance & Admin Services':
Group for department 'Fire Department':
Group for department 'Hearing Examiner':
Group for department 'Human Services Department':
Group for department 'Immigrant & Refugee Affairs':
Group for department 'Information Technology':
Group for department 'Intergovernment Relations':
Group for department 'Legislative Department':
Group for department 'Mayor's Office':
Group 

In [24]:
import pandas as pd
from sodapy import Socrata

client = Socrata("controllerdata.lacity.org", None)
results = client.get("g9h8-fvhu", limit=2000)

results_df = pd.DataFrame.from_records(results)

In [32]:
results_df[(results_df["employment_type"] != "PART_TIME") & (results_df["job_status"] != "NOT_ACTIVE")]

,record_nbr,pay_year,last_name,first_name,fms_dept,fms_department_title,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,regular_pay,overtime_pay,all_other_pay,total_pay,city_retirement_contributions,benefit_pay,gender,ethnicity
0,303030303233,2017,ESCOBEDO,JOSE,70,POLICE,2214-2,POLICE OFFICER II,FULL_TIME,ACTIVE,24,"POLICE OFFICERS, LIEUTENANT AND BELOW",91626.85,5835.35,2241.75,99703.95,27042.1,16819.0,MALE,HISPANIC
1,3030303032,2017,VILLEGAS,JOSE,44,LIBRARY,1172-1,LIBRARY ASSISTANT I,FULL_TIME,ACTIVE,3,CLERICAL,65332.56,0.0,0.0,65332.56,19377.64,7528.44,MALE,HISPANIC
3,3030303035,2017,BAUTISTA,RONALD,88,RECREATION AND PARKS,3141-0,GARDENER CARETAKER,FULL_TIME,ACTIVE,4,EQUIPMENT OPERATION AND LABOR,55628.8,2129.02,200.0,57957.82,16499.5,14946.17,MALE,HISPANIC
4,3030303037,2017,NUNEZ,ANGELICA,22,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT,9191-1,INDUSTRIAL COMMERCIAL FINANCE OFFICER I,FULL_TIME,ACTIVE,21,TECHNICAL,132912.0,0.0,350.0,133262.0,39421.7,6446.75,FEMALE,HISPANIC
7,3030303131,2017,MINETTA,VICTORIA,22,ECONOMIC AND WORKFORCE DEVELOPMENT DEPARTMENT,9207-0,HUMAN RELATIONS ADVOCATE,FULL_TIME,ACTIVE,1,ADMINISTRATIVE,92427.2,0.0,0.0,92427.2,27413.91,16777.89,FEMALE,CAUCASIAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,3033333233,2017,GONZALES,RUBEN,70,POLICE,2227-1,POLICE SERGEANT I,FULL_TIME,ACTIVE,24,"POLICE OFFICERS, LIEUTENANT AND BELOW",124529.56,6111.17,4138.26,134778.99,58342.1,14518.32,MALE,HISPANIC
1992,303333323633,2017,HA,MICHAEL,38,FIRE,2112-3,FIREFIGHTER III,FULL_TIME,ACTIVE,23,FIREFIGHTERS AND FIRE CAPTAINS,88179.6,69102.96,4322.98,161605.54,41312.14,9942.8,MALE,ASIAN
1993,303333323731,2017,SCOTT,ALBERT,82,PUBLIC WORKS - SANITATION,3844-0,INSTRUMENT MECHANIC SUPERVISOR,FULL_TIME,ACTIVE,9,PLANT EQUIPMENT OPERATION AND REPAIR,126292.95,2700.58,15951.9,144945.43,37458.49,17230.07,MALE,CAUCASIAN
1996,303333333033,2017,JUNEAU,JOSHUA,70,POLICE,2214-3,POLICE OFFICER III,FULL_TIME,ACTIVE,24,"POLICE OFFICERS, LIEUTENANT AND BELOW",101047.2,18294.04,2394.6,121735.84,47340.61,15515.38,MALE,BLACK


In [ ]:
# The correlation between Gender and Wage in the Los Angeles City Employee Payroll Database.
# The correlation between Ethnicity and Wage in the Los Angeles City Employee Payroll Database.
# Which are the departments with higher income in the Los Angeles City Employee Payroll Database?
# How do salary levels compare between equivalent departments in Los Angeles and Seattle, given their status as major West Coast cities with similar minimum wage rates?
# What does the wage's distribution in each department look like in the bar graph in the Los Angeles City Employee Payroll Database?